In [1]:
import sys
sys.path.insert(0, 'E:\Sebnewrepo\Fintech_ML_Lab_01')
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
from dataSource import data_source
from trade_Robot import trade_Robot
from wallet import wallet
from backtest import backtest

In [31]:
class strategy_01:
    def __init__(self, _basePrice, _param1, _initialCapital, _ticker):
        self.unit = _initialCapital/3
        self.x = _param1
        self.c = _basePrice
        self.y = 2*_param1/(2*_param1 + 3)
        self.buyline = self.c * (self.x + 1)
        self.stoploss = 0
        self.boughtIndex = 1   
        self.ds = data_source(_ticker)
        self.data = self.ds.get_Dailydata('2015-01-01', '2021-03-20')
        # initial ban interval is none
        self.low = 0
        self.high = 0
        #first test day must have no action, so position is none, but in minute level it will change
        self.no_position_index = -1 

    def get_Option(self, currentDate, currentWallet):
        ###todayPrice = ds[currentDate]
        # print('param transit success')
        # print('current date:' + currentDate)
        today = self.data.loc[self.data['date'] == currentDate]
        print('todays close price', max(today['close']))
        price = max(today['close'])
        print('new C', self.c)
        print('today stoploss', self.stoploss)
        print('today buyline', self.buyline)
        
        if max(currentWallet['currentPosition']) == 0:
            self.no_position_index += 1
            print('空仓', self.no_position_index)
            
        if self.no_position_index >= 14:
            print('too long time no holding position')
            self.update_Param(price)
            self.no_position_index = 0 

        if price <= self.stoploss and max(currentWallet['currentPosition']) != 0:
            print('sell all position')
            i = 0
            action = 'sell'
            volume = max(currentWallet['currentPosition'])
            print(price)
            self.ban_Interval_and_param_check(price)
            self.boughtIndex = 1
            print('current bought index', self.boughtIndex)
            self.set_Buyline()
            # reset no position index
            self.no_position_index = 0
            return 'sell', volume

        if price >= self.buyline:
            if self.boughtIndex == 1:
                action = 'buy'
                volume = self.unit // price
                self.boughtIndex += 1
                print('current bought index', self.boughtIndex)
                self.set_Buyline()
                self.set_Stoploss()
            elif self.boughtIndex > 1 and self.boughtIndex < 5:
                action = 'buy'
                volume = self.unit// 2 * price
                self.boughtIndex += 1
                print('add position not first time')
                print('current bought index', self.boughtIndex)
                self.set_Buyline()
                self.set_Stoploss()
            else:
                return 'no action', 0
        else:
            return 'no action', 0
        return action, volume

    def set_Buyline(self):
        self.buyline = self.c * np.power((self.x + 1),self.boughtIndex)
        return

    def set_Stoploss(self):
        self.stoploss = self.c * np.power((self.x + 1),(self.boughtIndex - 1)) * (1 - self.y)
        return
    
    def update_Param(self, new_price):
        print('update_parameter')
        self.c = new_price
        self.buyline = self.c * (self.x + 1)
        self.stoploss = self.c * (self.x + 1)*(1 - self.y)
        return
    
    def ban_Interval_and_param_check(self, price):
        self.low = self.stoploss
        self.boughtIndex += 1
        self.set_Buyline()
        self.high = self.buyline
        print('trade banning interval is:', self.low, 'to ', self.high)
        self.c = self.buyline / (self.x + 1)
        self.stoploss = self.c * (self.x + 1) * (1 - self.y)
        return
    

In [32]:
ymt = strategy_01(44.48, 0.07, 300000, 'TSLA')
ymt.y

0.0445859872611465

In [36]:
test_mt = backtest('TSLA', 300000, ymt, '2015-01-01', '2021-03-20')

In [37]:
test_mt.execute()

Back test Start, initial wallet stat:
        date ticker remainCapital currentPosition currentShareprice  \
0 2014-12-31   TSLA        300000               0                 0   

  currentSharesValue currentTotalCapital  
0                  0              300000  
-------------------------------------------
-----------------2014-12-31-----------------
todays close price 44.48
new C 44.48
today stoploss 0
today buyline 47.5936
空仓 0
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
        date ticker remainCapital currentPosition currentShareprice  \
1 2014-12-31   TSLA        300000               0             44.48   

  currentSharesValue currentTotalCapital  
1                  0              300000  
-----------------2015-01-02-----------------
todays close price 43.86
new C 44.48
today stoploss 0
today buyline 47.5936
空仓 1
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
        date ticker remainCapital currentPosition currentShare

wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
21 2015-01-30   TSLA        300000               0             40.72   

   currentSharesValue currentTotalCapital  
21                  0              300000  
-----------------2015-02-02-----------------
todays close price 42.19
new C 40.32
today stoploss 41.21885350318472
today buyline 43.1424
空仓 7
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
22 2015-02-02   TSLA        300000               0             42.19   

   currentSharesValue currentTotalCapital  
22                  0              300000  
-----------------2015-02-03-----------------
todays close price 43.67
new C 40.32
today stoploss 41.21885350318472
today buyline 43.1424
空仓 8
current bought index 2
action is:  buy
volume is:  2289.0
wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
23 2015-02-03   T

wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
40 2015-02-27   TSLA        292927               0             40.67   

   currentSharesValue currentTotalCapital  
40                  0              292927  
-----------------2015-03-02-----------------
todays close price 39.47
new C 46.162368
today stoploss 47.191465375796184
today buyline 49.39373376
空仓 11
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
41 2015-03-02   TSLA        292927               0             39.47   

   currentSharesValue currentTotalCapital  
41                  0              292927  
-----------------2015-03-03-----------------
todays close price 39.91
new C 46.162368
today stoploss 47.191465375796184
today buyline 49.39373376
空仓 12
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
         date ticker remainCapital currentPosition currentShareprice 

todays close price 43.89
new C 38.86
today stoploss 39.72630573248408
today buyline 44.490814
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
77 2015-04-22   TSLA        192961            2379             43.89   

   currentSharesValue currentTotalCapital  
77             104414              297376  
-----------------2015-04-23-----------------
todays close price 43.72
new C 38.86
today stoploss 39.72630573248408
today buyline 44.490814
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
         date ticker remainCapital currentPosition currentShareprice  \
78 2015-04-23   TSLA        192961            2379             43.72   

   currentSharesValue currentTotalCapital  
78             104010              296971  
-----------------2015-04-24-----------------
todays close price 43.69
new C 38.86
today stoploss 39.72630573248408
today buyline 44.490814
action is:  no a

113             119283              312244  
-----------------2015-06-15-----------------
todays close price 50.08
new C 38.86
today stoploss 48.6664327534395
today buyline 54.503160255002015
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
          date ticker remainCapital currentPosition currentShareprice  \
114 2015-06-15   TSLA        192961            2379             50.08   

    currentSharesValue currentTotalCapital  
114             119140              312102  
-----------------2015-06-16-----------------
todays close price 50.62
new C 38.86
today stoploss 48.6664327534395
today buyline 54.503160255002015
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
          date ticker remainCapital currentPosition currentShareprice  \
115 2015-06-16   TSLA        192961            2379             50.62   

    currentSharesValue currentTotalCapital  
115             120425              313386  
-----------------2015-06-17--------------

...

1473             702195              912184  
-----------------2020-11-05-----------------
todays close price 438.09
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1474 2020-11-05   TSLA        209989            1668            438.09   

     currentSharesValue currentTotalCapital  
1474             730734              940723  
-----------------2020-11-06-----------------
todays close price 429.95
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1475 2020-11-06   TSLA        209989            1668            429.95   

     currentSharesValue currentTotalCapital  
1475             717157              927146  
-----------------2020-11-09---

wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1508 2020-12-24   TSLA        209989            1668            661.77   

     currentSharesValue currentTotalCapital  
1508        1.10383e+06         1.31382e+06  
-----------------2020-12-28-----------------
todays close price 663.69
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1509 2020-12-28   TSLA        209989            1668            663.69   

     currentSharesValue currentTotalCapital  
1509        1.10703e+06         1.31702e+06  
-----------------2020-12-29-----------------
todays close price 665.99
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentS

wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1527 2021-01-25   TSLA        209989            1668             880.8   

     currentSharesValue currentTotalCapital  
1527        1.46917e+06         1.67916e+06  
-----------------2021-01-26-----------------
todays close price 883.09
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1528 2021-01-26   TSLA        209989            1668            883.09   

     currentSharesValue currentTotalCapital  
1528        1.47299e+06         1.68298e+06  
-----------------2021-01-27-----------------
todays close price 864.16
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentS

1546        1.19179e+06         1.40177e+06  
-----------------2021-02-23-----------------
todays close price 698.84
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1547 2021-02-23   TSLA        209989            1668            698.84   

     currentSharesValue currentTotalCapital  
1547        1.16567e+06         1.37565e+06  
-----------------2021-02-24-----------------
todays close price 742.02
new C 51.39
today stoploss 64.35841428716562
today buyline 72.07713344067302
action is:  no action
volume is:  0
lazy, no action today
wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1548 2021-02-24   TSLA        209989            1668            742.02   

     currentSharesValue currentTotalCapital  
1548        1.23769e+06         1.44768e+06  
-----------------2021-02-25---

wallet_updated
           date ticker remainCapital currentPosition currentShareprice  \
1565 2021-03-19   TSLA        209989            1668            654.87   

     currentSharesValue currentTotalCapital  
1565        1.09232e+06         1.30231e+06  
Last Day Reached, End
